## Домашка 4

как обычно, есть длинные выводы, мб удобнее клонить

In [2]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD, NMF
import gensim
import csv
import os
from sklearn.feature_extraction.text import CountVectorizer
import adagram
from collections import Counter
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import KFold
from scipy.spatial import distance
from sklearn.metrics import f1_score

In [36]:
from sklearn.metrics.pairwise import cosine_distances

In [3]:
train, test = pd.read_csv('paraphraser_train.csv'), pd.read_csv('paraphraser_test.csv')
# с диплома остался парафразер с очень тщательной нормализацией
print(train.head())
print(train.shape)
print(test.head())
print(test.shape)

   Unnamed: 0  Unnamed: 0.1  \
0           0             0   
1           1             1   
2           2             2   
3           3             3   
4           4             4   

                                       предложение 1  \
0  Полицейским разрешат стрелять на поражение по ...   
1  Право полицейских на проникновение в жилище ре...   
2  Президент Египта ввел чрезвычайное положение в...   
3  Вернувшихся из Сирии россиян волнует вопрос тр...   
4  В Москву из Сирии вернулись 2 самолета МЧС с р...   

                                       предложение 2  класс  \
0  Полиции могут разрешить стрелять по хулиганам ...      0   
1  Правила внесудебного проникновения полицейских...      0   
2  Власти Египта угрожают ввести в стране чрезвыч...      0   
3  Самолеты МЧС вывезут россиян из разрушенной Си...     -1   
4  Самолеты МЧС вывезут россиян из разрушенной Си...      0   

                                             леммы 1  \
0  полицейский разрешать стрелять поражен

In [7]:
# чтение одного мистического корпуса
sentences = []
sentences_raw = []
for file in os.listdir(os.path.join(os.getcwd(),'corpus')):
    with open('corpus/' + file, 'r', encoding='utf-8') as  f:
        sentence = []
        reader = csv.reader(f, delimiter=';')
        for line in reader:
            word = line[2]
            if word.isalpha():
                sentence.append(word.lower())
        sentences.append(sentence)
        sentences_raw.append(' '.join(sentence))

In [30]:
#print(sentences[0][:20])

['год', 'на', 'ямал', 'более', 'или', 'менее', 'нормальный', 'погода', 'держаться', 'в', 'сочи', 'несколько', 'день', 'начинать', 'портиться', 'связанный', 'это', 'с', 'прохождение', 'атмосферный']


In [8]:
cv = CountVectorizer( max_features=1000)
X = cv.fit_transform(sentences_raw)

In [37]:
svd = TruncatedSVD(50)
svd.fit(X)
id2word = {i:w for i,w in enumerate(cv.get_feature_names())}
word2id = {w:i for i,w in id2word.items()}
id2vec_svd = svd.components_.T
def most_similar(word, id2vec):
    similar = [id2word[i] for i in cosine_distances(id2vec[word2id[word]].reshape(1, -1), id2vec).argsort()[0][:10]]
    return similar
print(most_similar('погода', id2vec_svd))

['погода', 'мчс', 'снежный', 'пожарный', 'столица', 'госпитализировать', 'температура', 'чп', 'сообщение', 'встречный']


In [38]:
nmf = NMF(6)
nmf.fit(X)
id2vec_nmf = nmf.components_.T
print(most_similar('погода', id2vec_nmf))

['погода', 'мчс', 'снежный', 'ожидаться', 'полоса', 'ночь', 'пожарный', 'секунда', 'трасса', 'аэропорт']


In [20]:
# fast_text = gensim.models.FastText(sentences, min_n=1)
# fast_text.save('fasttext')

In [21]:
# fast_text.wv.most_similar('погода')

[('подмога', 0.9999983310699463),
 ('пахомов', 0.9999969005584717),
 ('пожаро', 0.9999966621398926),
 ('выгода', 0.9999966025352478),
 ('автодром', 0.999996542930603),
 ('автодорога', 0.9999964237213135),
 ('автопром', 0.9999961853027344),
 ('вода', 0.9999960064888),
 ('автодор', 0.9999960064888),
 ('ягода', 0.9999960064888)]

In [23]:
# w2v = gensim.models.Word2Vec(sentences, size=50, sg=1)
# w2v.save('w2v')

In [28]:
# w2v.wv.most_similar('мчс')

[('областной', 0.9963815212249756),
 ('региональный', 0.9956607818603516),
 ('прокуратура', 0.9955354332923889),
 ('администрация', 0.9954781532287598),
 ('брянский', 0.9953436851501465),
 ('псковский', 0.9945963025093079),
 ('су', 0.993961751461029),
 ('правительство', 0.9921631813049316),
 ('край', 0.9914276003837585),
 ('информация', 0.991313636302948)]

In [9]:
newtrain = pd.concat([train,test]).dropna()# возьмем все, а потом сделаем к фолд
cv_train1 = cv.transform(newtrain['леммы 1'])
cv_train2 = cv.transform(newtrain['леммы 2']) 

In [39]:
def count_cosine(model,cv_train1,cv_train2):
    out = []
    for sent1, sent2 in zip(model.transform(cv_train1), model.transform(cv_train2)):
        out.append(distance.cosine(sent1,sent2))
    return out

In [40]:
svd_feature = count_cosine(svd,cv_train1,cv_train2)
nmf_feature = count_cosine(nmf,cv_train1,cv_train2)

/home/laks/compling_nlp_hse_course/env/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/home/laks/compling_nlp_hse_course/env/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [41]:
w2v = gensim.models.Word2Vec.load('w2v')
fast_text = gensim.models.FastText.load('fasttext')

In [42]:
def get_embedding(text, model, dim):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [43]:
dim = 50
X_text_1_w2v = np.zeros((len(newtrain['леммы 1']), dim))
X_text_2_w2v = np.zeros((len(newtrain['леммы 2']), dim))

for i, text in enumerate(newtrain['леммы 1']):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(newtrain['леммы 2']):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)
                         
w2v_feature = []
for sent1, sent2 in zip(X_text_1_w2v,X_text_2_w2v):
    w2v_feature.append(distance.cosine(sent1,sent2))

/home/laks/compling_nlp_hse_course/env/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [44]:
dim = 100
X_text_1_fast_text = np.zeros((len(newtrain['леммы 1']), dim))
X_text_2_fast_text = np.zeros((len(newtrain['леммы 2']), dim))

for i, text in enumerate(newtrain['леммы 1']):
    X_text_1_fast_text[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(newtrain['леммы 2']):
    X_text_2_fast_text[i] = get_embedding(text, fast_text, dim)
                         
fast_text_feature = []
for sent1, sent2 in zip(X_text_1_fast_text,X_text_2_fast_text):
    fast_text_feature.append(distance.cosine(sent1,sent2))

/home/laks/compling_nlp_hse_course/env/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [4]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [45]:
def get_embedding_adagram(text, model, window, dim):
    text = text.split()
    
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))
    
    
    
    vectors = np.zeros((len(word2context), dim))
    for i,word in enumerate(word2context):
        word, context = word
        try:
            
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v 
            
        except (KeyError, ValueError):

            continue
            
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
dim = 300
X_text_1_adagram = np.zeros((len(newtrain['леммы 1']), dim))
X_text_2_adagram = np.zeros((len(newtrain['леммы 2']), dim))

for i, text in enumerate(newtrain['леммы 1']):
    X_text_1_adagram[i] = get_embedding_adagram(text, vm, 5, dim)
    
for i, text in enumerate(newtrain['леммы 2']):
    X_text_2_adagram[i] = get_embedding_adagram(text, vm, 5, dim)

adagram_feature = []
for sent1, sent2 in zip(X_text_1_adagram,X_text_2_adagram):
    adagram_feature.append(distance.cosine(sent1,sent2))

/home/laks/compling_nlp_hse_course/env/lib/python3.6/site-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


In [13]:
vm.sense_neighbors('россия', 0) # для парафразов кстати не очень хорошо, 
#хотя эти слова и объединяются по смыслу,
#предложения отличающиеся только на россию и америку не будут парафразом
#а модель скажет что будут

/home/laks/compling_nlp_hse_course/env/lib/python3.6/site-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


[('ссср', 9, 0.69153535),
 ('европа', 1, 0.6867954),
 ('украина', 6, 0.6639078),
 ('эстония', 3, 0.65575594),
 ('америка', 1, 0.6406993),
 ('беларусь', 3, 0.6262512),
 ('германия', 5, 0.61813647),
 ('латвия', 5, 0.60231125),
 ('прибалтика', 1, 0.5997372),
 ('казахстан', 4, 0.5986538)]

In [46]:
print(adagram_feature[:20])

[0.3182581391691238, 0.21601673456380055, 0.14326977013679187, 0.6751250999135716, 0.3778728585359328, 0.18230417825351286, 0.4855077050675436, 0.2191455671226039, 0.7159037068924574, 0.7274834416859948, 0.2656663757119202, 0.8475680041307854, 0.5711972000293315, 0.1289074725920718, 0.3019134429374072, 0.32649542725222147, 0.4963361216936796, 0.41648312599438797, 0.40073438216410984, 0.25532351910373574]


In [47]:
X = pd.DataFrame({'fasttext':fast_text_feature,
                  'w2v':w2v_feature,
                  'svd':svd_feature,
                  'nmf':nmf_feature,
                 'adagram':adagram_feature}, index=[i for i in range(9150)]).fillna(0)
y= newtrain['класс']

In [48]:
kf = KFold(n_splits=5)
f1_scores = []
for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf = RandomForestClassifier(n_estimators=200,
                             class_weight='balanced')
    
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    f1_scores.append(f1_score(y_test, preds, average='micro'))
    
print(np.mean(f1_scores)) 

0.5083060109289617


## Как улучшить

* вместо эмбеддингов брать метрики по RuWordNet (улучшится ненамного, и не хочется повторять диплом)
* объединить парафраз с частичным парафразом
* разметить например UDPipe и считать схожесть между словами с одинаковыми синтаксическими тегами

In [5]:
# # вот отсюда https://github.com/ufal/udpipe/blob/master/bindings/python/examples/udpipe_model.py
# import ufal.udpipe 

# class Model:
#     def __init__(self, path):
#         """Load given model."""
#         self.model = ufal.udpipe.Model.load(path)
#         if not self.model:
#             raise Exception("Cannot load UDPipe model from file '%s'" % path)

#     def tokenize(self, text):
#         """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
#         tokenizer = self.model.newTokenizer(self.model.DEFAULT)
#         if not tokenizer:
#             raise Exception("The model does not have a tokenizer")
#         return self._read(text, tokenizer)

#     def read(self, text, in_format):
#         """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
#         input_format = ufal.udpipe.InputFormat.newInputFormat(in_format)
#         if not input_format:
#             raise Exception("Cannot create input format '%s'" % in_format)
#         return self._read(text, input_format)

#     def _read(self, text, input_format):
#         input_format.setText(text)
#         error = ufal.udpipe.ProcessingError()
#         sentences = []

#         sentence = ufal.udpipe.Sentence()
#         while input_format.nextSentence(sentence, error):
#             sentences.append(sentence)
#             sentence = ufal.udpipe.Sentence()
#         if error.occurred():
#             raise Exception(error.message)

#         return sentences

#     def tag(self, sentence):
#         """Tag the given ufal.udpipe.Sentence (inplace)."""
#         self.model.tag(sentence, self.model.DEFAULT)

#     def parse(self, sentence):
#         """Parse the given ufal.udpipe.Sentence (inplace)."""
#         self.model.parse(sentence, self.model.DEFAULT)

#     def write(self, sentences, out_format):
#         """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

#         output_format = ufal.udpipe.OutputFormat.newOutputFormat(out_format)
#         output = ''
#         for sentence in sentences:
#             output += output_format.writeSentence(sentence)
#         output += output_format.finishDocument()

#         return output

In [12]:
# model = Model('russian-syntagrus-ud-2.0-170801.udpipe')
# sent1, sent2 = [], []

# def to_conllu(sent, out):
#     for tr in sent:
#         sentences = model.tokenize(tr)
#         for s in sentences:
#             model.tag(s)
#             model.parse(s)
#         conllu = model.write(sentences, "conllu")
#         out.append(conllu)

# to_conllu(train['предложение 1'], sent1)
# to_conllu(train['предложение 2'], sent2)
# train.loc[:,'syntax1']=pd.Series(sent1)
# train.loc[:,'syntax2']=pd.Series(sent2)
# train.to_csv('paraphraser_train.csv') # парсинг работает быстро, но не совсем, поэтому сохраним
# sent1, sent2 = [], []
# to_conllu(test['предложение 1'], sent1)
# to_conllu(test['предложение 2'], sent2)
# test.loc[:,'syntax1']=pd.Series(sent1)
# test.loc[:,'syntax2']=pd.Series(sent2)
# test.to_csv('paraphraser_test.csv')
# print(train.head())

   Unnamed: 0                                      предложение 1  \
0           0  Полицейским разрешат стрелять на поражение по ...   
1           1  Право полицейских на проникновение в жилище ре...   
2           2  Президент Египта ввел чрезвычайное положение в...   
3           3  Вернувшихся из Сирии россиян волнует вопрос тр...   
4           4  В Москву из Сирии вернулись 2 самолета МЧС с р...   

                                       предложение 2  класс  \
0  Полиции могут разрешить стрелять по хулиганам ...      0   
1  Правила внесудебного проникновения полицейских...      0   
2  Власти Египта угрожают ввести в стране чрезвыч...      0   
3  Самолеты МЧС вывезут россиян из разрушенной Си...     -1   
4  Самолеты МЧС вывезут россиян из разрушенной Си...      0   

                                             леммы 1  \
0  полицейский разрешать стрелять поражение гражд...   
1  право полицейский проникновение жилище решать ...   
2  президент египет вводить чрезвычайный пол

In [73]:
from conllu import parse


In [84]:
tree=parse(newtrain.iloc[1]['syntax1'])
print(tree[0][0])

OrderedDict([('id', 1), ('form', 'Право'), ('lemma', 'право'), ('upostag', 'NOUN'), ('xpostag', None), ('feats', OrderedDict([('Animacy', 'Inan'), ('Case', 'Nom'), ('Gender', 'Neut'), ('Number', 'Sing')])), ('head', 7), ('deprel', 'nsubj'), ('deps', None), ('misc', None)])


In [99]:
roots1,roots2=[],[]
for i in range(newtrain.shape[0]):
    tree1=parse(newtrain.iloc[i]['syntax1'])
    tree2=parse(newtrain.iloc[i]['syntax2'])
    root1, root2 = '',''
    for x in tree1[0]:
        if x['deprel']=='root':
            root1 = x['lemma']
        break
    for x in tree2[0]:
        if x['deprel']=='root':
            root2 = x['lemma']
        break
    roots1.append(root1)
    roots2.append(root2)
for r1, r2 in zip(roots1,roots2):
    print([r1],[r2]) #много где руты не определены
    #но там где определены, выглядят довольно разумно, например, скончаться-умирать,уходить- скончаться
    #президент - Пески это шикарно
    #морфоанализаторы ваще его не любят, в томитовском майстеме он Песок

[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['скончаться']
[''] ['']
[''] ['']
['лдпр'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['взрыв'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['пойти'] ['']
[''] ['']
[''] ['']
['умирать'] ['умирать']
[''] ['']
[''] ['']
[''] ['']
['пожар'] ['']
[''] ['']
[''] ['']
['сми'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['пожар'] ['']
[''] ['']
[''] ['']
['Генконсул'] ['становиться']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['возбуждать'] ['']
['скончаться'] ['скончаться']
[''] ['']
[''] ['']
[''] ['']
[''] ['']

[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['скончаться'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['миллиардер'] ['']
[''] ['']
[''] ['']
[''] ['']
['скончаться'] ['']
[''] ['виктор']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['называть']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['обнародовать'] ['опубликовывать']
[''] ['']


[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['опубликовывать']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['сми'] ['']
[''] ['']
[''] ['']
[''] ['']
['выборы'] ['']
[''] ['технология']
[''] ['']
[''] ['']
[''] ['']
[''] ['идти']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['минфин'] ['лекарство']
[''] ['открываться']
['сборная'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['мария'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['открываться']
['владимир'] ['']
[''] ['']
[''] ['кумир']
[''] ['']
['аркадий'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['Форт']
[''] ['блеск']
[''] ['']
[''] ['']
[''] ['']
['эксперт'] ['долг']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['«Вопрос']
[''] ['']
[''] ['']
[''] ['

[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['вспышка'] ['']
[''] ['']
['Генконсул'] ['']
['скончаться'] ['умирать']
[''] ['']
['сми'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Мосгоризбирк'] ['']
[''] ['']
['израиль'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['умирать']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['погибать']
[''] ['']
[''] ['']
[''] ['']
[''] ['умирать']
[''] ['']
['Предполагать'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['скончаться'] ['скончаться']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['ЕСПЧ']
[''] ['']
[''] ['']
[''] ['']
[''] ['определяться']
[''] ['']
[''] 

[''] ['граница']
[''] ['']
[''] ['«Вопрос']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['игорь']
[''] ['']
[''] ['']
[''] ['беспорядок']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['начинать']
['Госпитализированный'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['скончаться']
[''] ['']
[''] ['']
['опубликовывать'] ['обнаруживать']
[''] ['обнаруживать']
[''] ['']
[''] ['']
[''] ['']
['скончаться'] ['скончаться']
[''] ['']
[''] ['скончаться']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['бывший']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['завершать']
[''] ['']
[''] ['']
[''] ['обнаруживать']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['начинать']
[''] ['обнаруживать']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']

...

[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['врач'] ['']
['скворцова'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['россельхознадзор'] ['']
[''] ['']
[''] ['']
['Силуаен'] ['']
[''] ['']
['сми'] ['']
['путин'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['россельхознадзор'] ['Россельхоз']
['командующий'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['шойгу'] ['сергей']
[''] ['']
[''] ['']
[''] ['']
['шойгу'] ['шойгу']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['рфс'] ['']
[''] ['']
['рфс'] ['']
['сми'] ['']
[''] ['']
[''] ['']
['Порошенко'] ['']
['иванов'] ['сергей']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Рябк'] ['']
['Рябк'] ['']
['Рябк'] ['']
[''] ['']
['сми'] ['Хуг']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['посетитель'] ['']
['видео'] ['']
['Ск'] ['']
['Чуркин'] ['Чуркин']
[''] ['']
[

[''] ['']
['сафронов'] ['']
[''] ['']
['Ъ'] ['']
[''] ['']
[''] ['']
[''] ['']
['Ъ'] ['']
[''] ['']
[''] ['']
[''] ['']
['Мединский'] ['']
['Мединский'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Ъ'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Пески'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Ъ'] ['']
[''] ['']
[''] ['']
['осуждать'] ['']
[''] ['']
[''] ['']
[''] ['']
['ГП'] ['']
[''] ['']
['ГП'] ['']
[''] ['']
['главврач'] ['']
[''] ['']
['роспотребнадзор'] ['Роспотребнадор']
['радиационный'] ['Роспотребнадор']
[''] ['']
[''] ['']
['Нас'] ['']
[''] ['']
[''] ['']
[''] ['']
['ФСИН'] ['']
[''] ['']
['ФСИН'] ['']
['адвокат'] ['']
[''] ['']
['адвокат'] ['']
['осуждать'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['ДНР'] ['ДНР']
['ДНР'] ['ДНР']
[''] ['']
[''] ['']
['скончаться'] ['скончаться']
[''] ['']
[''] ['']
['скончаться'] ['']
['мид'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['полиция'] ['']
[''] 

[''] ['']
['день'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Рябк'] ['']
[''] ['']
[''] ['']
[''] ['']
['греф'] ['']
['греф'] ['']
[''] ['']
['день'] ['']
[''] ['Стивенный']
[''] ['Стивенный']
[''] ['Стивенный']
[''] ['Стивенный']
[''] ['Стивенный']
[''] ['']
[''] ['']
['оон'] ['']
['мвд'] ['']
['Москаль'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['греф'] ['']
[''] ['']
[''] ['']
[''] ['']
['путин'] ['путин']
['президент'] ['путин']
[''] ['']
[''] ['']
['мвд'] ['']
[''] ['']
['Пургин'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['день'] ['']
[''] ['']
[''] ['путин']
['путин'] ['путин']
['президент'] ['путин']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['представитель']
['адмирал'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['мвд'] ['']
[''] ['']
['Рогозин'] ['']
[''] ['генсек']
['мвд'] ['']
[''] ['']
['мид'] ['']
[''] ['']
['оон'] ['']
[''] ['']
[''] ['']
[''] ['']
['генсек'] ['']
[

[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['устанавливать']
[''] ['']
[''] ['']
['умирать'] ['скончаться']
[''] ['']
[''] ['']
['скончаться'] ['уходить']
[''] ['']
[''] ['']
[''] ['']
['пригожин'] ['']
['пригожин'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Роструд'] ['Роструд']
[''] ['']
['Сб'] ['']
[''] ['']
[''] ['']
['сми'] ['']
[''] ['']
['сми'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['сми'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Ъ'] ['']
['сб'] ['']
['Турчин'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['Порошенко']
[''] ['']
[''] ['']
[''] ['']
['Пески'] ['']
[''] ['']
['сми'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Ск'] ['']
[''] ['']
['Ск'] ['']
[''] ['']
[''] ['']
['Ск'] ['']
['ОБСЕ'] ['']
[''] ['']
[''] ['']
[''] ['']
['улюкаев'] ['']
['путин'] ['']
['Il'] ['']
[''] ['']
[''] ['']
[''] ['россельхознадзор']
[''] [

[''] ['']
[''] ['']
[''] ['']
[''] ['']
['аэропорт'] ['']
[''] ['']
[''] ['']
[''] ['']
['источник'] ['источник']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['сми'] ['']
[''] ['']
[''] ['']
['Винокуров'] ['эксперт']
[''] ['']
['сми'] ['']
['Кадыр'] ['']
['сми'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Ск'] ['']
[''] ['']
[''] ['']
[''] ['']
['Коломойский'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Ск'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['источник'] ['']
['Ск'] ['Ск']
[''] ['']
[''] ['']
[''] ['']
['сми'] ['']
[''] ['']
[''] ['']
['медведев'] ['']
[''] ['']
[''] ['']
[''] ['']
['Аракча'] ['']
['источник'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['']
['Пушек'] ['']
[''] ['']
[''] ['']
[''] ['']
[''] ['washington']
[''] ['']


In [101]:
cv_root1 = cv.transform(roots1)
cv_root2 = cv.transform(roots2)
svd_root = count_cosine(svd,cv_root1,cv_root2)
nmf_root = count_cosine(nmf,cv_root1,cv_root2)
print(svd_root) #очень много nan, поэтому может ничего и не получиться

/home/laks/compling_nlp_hse_course/env/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1.0138426085134313, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [102]:
X['svd_root']=pd.Series(svd_root).fillna(0.0)
X['nmf_root']=pd.Series(nmf_root).fillna(0.0)

In [103]:
kf = KFold(n_splits=5)
f1_scores = []
for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf = RandomForestClassifier(n_estimators=200,
                             class_weight='balanced')
    
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    f1_scores.append(f1_score(y_test, preds, average='micro'))
    
print(np.mean(f1_scores)) 

0.507103825136612


как и следовало ожидать, качество немного упало.
Чтобы оно росло, синтаксические признаки надо строить так, чтобы нулевые значения были только там, где действительно не парафразы.
То есть использовать не только root.
Но это на еще один диплом задача хд